In [67]:
import pandas as pd  
import requests
import datetime as dt

In [175]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'Accept': 'application/json',
    'Referer': 'https://zh.vacme.ch/',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ***find me inbrowser netwrk tab**',
    'Origin': 'https://zh.vacme.ch',
}

registration_id = '*** find me inbrowser netwrk tab ***'

In [176]:
def doRequestFirstPerId(id):
    resp = requests.post('https://zh.vacme.ch/api/v1/reg/dossier/termine/nextfrei/%s/ERSTE_IMPFUNG' % id, headers=headers)
    return resp.json() if resp.text != '' else ''

def doRequestSecondPerIdAndStart(id, nextDate):
    data = '{"nextDate": "%s"}' % nextDate
    resp = requests.post('https://zh.vacme.ch/api/v1/reg/dossier/termine/nextfrei/%s/ZWEITE_IMPFUNG' % id, headers=headers, data=data)
    return resp.json() if resp.text != '' else ''

def doListLocations():
    resp = requests.get('https://zh.vacme.ch/api/v1/reg/dossier/odi/all/%s' % registration_id, headers=headers)  
    return resp.json() if resp.text != '' else ''

In [177]:
locations = doListLocations()
print("found %s locations" % len(locations))

found 0 locations


In [167]:
nextFirstDateLocations = []

for location in locations:
  #print("requesting %s %s" % (location['id'], location['name']))
  resp = doRequestFirstPerId(location['id'])
  if resp != '':
    #print(location['name'] + ": " + resp)
    nextFirstDateLocations.append({
        'locationId': location['id'],
        'name' : location['name'],
        'nextDate': resp['nextDate'],
        'nextDateParsed': dt.datetime.strptime(resp['nextDate'], '%Y-%m-%dT%H:%M:%S')
    })
    
nextFirstDateLocations.sort(key=lambda x: x['nextDateParsed'])
len(nextFirstDateLocations)

19

In [168]:
for next in nextFirstDateLocations:
    print(next['nextDateParsed'].strftime("%Y-%m-%d") + " " + next['name'])

2021-05-18 Zürich, Medbase Apotheke Zürich Enge
2021-05-21 Geroldswil, Pill Apotheke im Dorf
2021-05-22 Meilen, TopPharm Apotheke & Parfumerie Dr. Rebhan AG
2021-05-27 Schlieren, Apotheke Dr Kunz Schlieren
2021-05-30 Zürich, Victoria Apotheke im Circle
2021-05-31 Regensdorf, Toppharm Zentrums Apotheke
2021-06-02 Zollikon, Apotheke Zollikon
2021-06-03 Horgen, Toppharm Apotheke zum Erzberg A&A medical AG
2021-06-03 _Impfzentrum Wetzikon
2021-06-05 Zürich, TopPharm Limmatplatz Apotheke
2021-06-08 _Impfzentrum Triemli Zürich
2021-06-08 _Impfzentrum Messe Zürich
2021-06-08 Zürich, Medbase Apotheke Zürich Industrie
2021-06-09 Zürich, Medbase Apotheke Zürich Nord
2021-06-10 Winterthur, Rathaus Apotheke AG
2021-06-14 Zürich, Apotheke Leimbach
2021-06-15 Winterthur, Toppharm Apotheke Wülflingen AG
2021-07-02 Uster, Dropa Drogerie Apotheke Illuster
2021-07-30 Winterthur, Bahnhof Apotheke


In [169]:
nextSecondDateLocations = []
for next in nextFirstDateLocations:
    resp = doRequestSecondPerIdAndStart(next['locationId'], next['nextDate'])
    if resp != '':
        available = next.copy()
        available['secondDate'] = resp['nextDate']
        print(next['name'] + " " + next['name'])
        nextSecondDateLocations.append(available)

_Impfzentrum Triemli Zürich _Impfzentrum Triemli Zürich
_Impfzentrum Messe Zürich _Impfzentrum Messe Zürich


In [170]:
nextSecondDateLocations

[{'locationId': 'a16d2593-0c27-4a62-a471-0ca9ad98536c',
  'name': '_Impfzentrum Triemli Zürich',
  'nextDate': '2021-06-08T12:00:00',
  'nextDateParsed': datetime.datetime(2021, 6, 8, 12, 0),
  'secondDate': '2021-07-06T12:00:00'},
 {'locationId': 'b0893fdc-4e53-429e-a8a9-9b67f8159d60',
  'name': '_Impfzentrum Messe Zürich',
  'nextDate': '2021-06-08T12:00:00',
  'nextDateParsed': datetime.datetime(2021, 6, 8, 12, 0),
  'secondDate': '2021-07-06T12:00:00'}]